In [1]:
#RUN ME BEFORE ANYTHING ELSE!!! I import all of the required modules and functions!!
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()
print('Imported and initialised matplotlib...')
import astropy.io.fits as ast
print('Imported Astropy...')
from MapFunc import *
print('Imported User Defined Functions...')
from tools.mapping import *
from tools.image import *
print("Imported Oliver King's tools, v1.4 ...")
col = 'viridis'
xcol = 'red'
lvls = 1000
print("Graphical Parameters initialised...")
print('All Modules Imported Successfully!')

Imported and initialised matplotlib...
Imported Astropy...
Imported User Defined Functions...
Imported Oliver King's tools, v1.4 ...
Graphical Parameters initialised...
All Modules Imported Successfully!


# 2008 467nm

In [ ]:
mfac = 0.88
img, hdr, scihdr = open_img('data/2008/467/hst_11156_47_wfpc2_f467m_pc_drz.fits')
eph = get_ephemerides(target='neptune',loc='hst',epoch=hdr['EXPSTART'])

IF, pltscl = calc_IF(eph['r'], scihdr['PHOTFLAM'], 2019.07, PlateScale = 0.05)

img = interp_image(img,4) #interpolate image to have higher resolution, rest of the block deals with the consequences of this
pltscl = pltscl/4

plt.figure(figsize=(15,8))
ax1 = plt.subplot(221)
im = ax1.imshow(img, cmap=col)
plt.title('Raw MAST Image')
plt.colorbar(im)
ax1.axis('off')

img = img*IF

ax2 = plt.subplot(222)
im2 = ax2.imshow(img, cmap=col)
plt.title('Calibrated Image')
plt.colorbar(im2)
ax2.axis('off')

rad = float(eph['ang_width'])/(2*pltscl)
img, y, x = circ(img,rad)
img, planicentre = get_planicentre(img,rad)

ax3 = plt.subplot(223)
im3 = ax3.imshow(img, cmap=col)
plt.title('Zoomed Image with Planicentre')
ax3.axhline(planicentre[1],color=xcol)
ax3.axvline(planicentre[0],color=xcol)
ax3.plot(x+planicentre[0], y+planicentre[1],color=xcol)
plt.colorbar(im3)
ax3.axis('off')

img = minnaert(img, planicentre, mfac, eph)

ax4 = plt.subplot(224)
im4 = ax4.imshow(img, cmap=col)
plt.title('Minnaert Corrected Image')
#ax4.axhline(planicentre[1],color=xcol)
#ax4.axvline(planicentre[0],color=xcol)
ax4.plot(x+planicentre[0], y+planicentre[1],color=xcol)
plt.colorbar(im4)
ax4.axis('off')

ang_corr = float(scihdr['ORIENTAT'])-float(eph['NPole_ang']) #the angle with which to correct the north pole of the image

mapped_img = map_observation(img, planicentre[0], planicentre[1], planicentre[2], ang_corr, eph['PDObsLon'], eph['PDObsLat'])

plt.figure(figsize=(15,7))
plt.contourf(mapped_img[1], mapped_img[2], mapped_img[0],levels=lvls)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('2008 Map: F467M. Minnaert Correction = '+str(mfac))
plt.colorbar()
plt.show()

flipped_img = np.flip(mapped_img[0],0)

#plt.figure(figsize=(15,14))
#ax1 = plt.subplot(211)
#ax1.imshow(mapped_img[0])
#ax2 = plt.subplot(212)
#ax2.imshow(flipped_img)
#plt.show()

#Save the Plot to a File
plt.figure(figsize=(15,7)) #saves the map to a .png to possibly be used in my report
plt.contourf(mapped_img[1], mapped_img[2], mapped_img[0],levels=lvls)
plt.axis('off')
plt.savefig('maps/2008/467_1.png')
plt.close()

hdu = ast.PrimaryHDU(flipped_img) #saves the map to a fits image to be read by the reflectance plotter#
hdu.writeto('maps/2008/467_cal.fits')

writer467.writerow([float(hdr['EXPSTART'])])